##### Copyright 2023 Krzysztof Rusek
AGH University of Science and Technology
Licensed under the Apache License, Version 2.0 (the "License");


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode:"form" }
#   Copyright 2023 Krzysztof Rusek
#
#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.


# Queuing Theory
This notebook shows how to apply queuing theory model on a custom network.


In [ ]:
import numpy as np
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import random
import queuinx as qx

from itertools import pairwise
from functools import partial
from jaxopt import FixedPointIteration

In [ ]:
sns.set()

# Model



# Network

In this notebook, we are going to predict the delay in a random network with shortest path routing. Let's use Barabasi-Alber small world model of a network with $n$ nodes. Python package `networkx` implements generators and shortest path computation routines.

The traffic matrix is randomly generated from a uniform distribution : $TM\sim Uni(0.1,1.1)$ that resembles demands used in the simulation. 
Note that we assume constant link speed the same as described in the paper.

In [ ]:
#@title Simulation parameters { display-mode:"form" }
#@markdown Number of nodes in the network
n=10 #@param 

G = nx.barabasi_albert_graph(n,2)
G = nx.DiGraph(G)

nx.draw_networkx(G)


Find the shortest paths

In [ ]:
all_pairs=dict(nx.all_pairs_dijkstra_path(G))
edges=list(G.edges)


In `networkx` edges are defined by the pair of endpoints. In RouteNetStep, the network is defined with links in mind, thus we need to convert the presentation and create indices used as an input of RouteNetStep.

Routing tensor

In [ ]:
edge_index = {e: i for i, e in enumerate(edges)}

routing = []

for i in range(n):
    for j in range(n):
        if i != j:
            path = all_pairs[i][j]
            routing.append(list(pairwise(path)))
max_len = max(map(len, routing))
R = np.zeros(shape=(n * (n - 1), max_len, len(edges)))

for i, p in enumerate(routing):
    for j, e in enumerate(p):
        k = edge_index[e]
        R[i, j, k] = 1 #F,S,Q


## Traffic

In [ ]:
BOUNDARIES = [(8,10), (10,12), (12,14), (14,16), (16,18)]
rho_l = 0.2 #@param
rho_h = 0.9 #@param

boundaries = random.choice(BOUNDARIES)
TM = np.random.uniform(boundaries[0], boundaries[1], size=n * (n - 1)) * np.random.uniform(low=0.1,
                                                                                        size=n * (n - 1)) / (n - 1)
traffic_noloss = np.tensordot(R, TM, axes=(0, 0)).sum(axis=0)

rho = np.random.uniform(rho_l, rho_h, size=len(edges))

Pb = np.zeros_like(rho)
mu = traffic_noloss / rho
buffer_upper_bound=64

In [ ]:
tm = qx.PoissonFlow(rate=TM)

q = qx.FiniteFifo(
    service_rate=jnp.asarray(mu, dtype=np.float32),
    b=32. + jnp.zeros(mu.shape, dtype=np.float32),
    arrivals=jnp.zeros(mu.shape, dtype=np.float32),
    pasprob=jnp.ones(mu.shape)
)

flow, step, queue = jnp.where(R)
n_flow, n_step, n_queue = jax.tree_map(lambda x: jnp.asarray(x), R.shape)
n_route = jnp.array([len(flow)])

net = qx.Network(queues=q, flows=tm, flow=flow, step=step, queue=queue, n_flows=n_flow[..., jnp.newaxis],
            max_path_length_mask=jnp.ones((1, n_step)), n_queues=n_queue[..., jnp.newaxis], n_routes=n_route)


In [ ]:
def fixedpoint(net: qx.Network) -> qx.Network:
    model = qx.FiniteApproximationJackson(buffer_upper_bound)

    def T(x, params: qx.Network):
        params = params.replace(queues=params.queues.update_dynamic_fields(x))
        y = model(params)
        return y.queues.get_dynamic_fields()

    fpi = FixedPointIteration(fixed_point_fun=T, maxiter=500, jit=True)

    opt = fpi.run(net.queues.get_dynamic_fields(), net)
    return net.replace(queues=net.queues.update_dynamic_fields(opt.params))


In [ ]:
@jax.jit
def _sol(x):
    fix = fixedpoint(x)
    qos_pass = qx.Readout_mm1b(buffer_upper_bound)
    qos = qos_pass(fix)
    return qos


# Visualisation

In [ ]:
fix_qos = _sol(net)
sns.histplot(fix_qos.flows.delay)

Let's scale TM and plot the average delay.
For small load, we expect the delay to be equal to propagation time.

For efficiency we compile the whole function.

Note how `jax.vmap` helps computing solution for multiple points


In [ ]:
@jax.jit
@partial(jax.vmap, in_axes=(None,0))
def _sol_2(x,scale):
    x = x.replace(flows=qx.PoissonFlow(rate=scale*x.flows.rate))
    fix = fixedpoint(x)
    qos_pass = qx.Readout_mm1b(buffer_upper_bound)
    qos = qos_pass(fix)
    model = qx.FiniteApproximationJackson(buffer_upper_bound)

    throughput = model(fix).flows.rate # rate at the end

    # average delay weighted by the intensity
    w = jnp.dot(throughput,qos.flows.delay)/throughput.sum()

    return w



In [ ]:
i = jnp.arange(0.001,2.,step=0.01)
ws = _sol_2(net,i)
sns.lineplot(x=i,y=ws)
plt.xlabel('scale')
plt.ylabel('average delay')

# References

F. P. Kelly. 2011. Reversibility and Stochastic Networks. Cambridge University Press, New York, NY, USA.